# Working with data frames continued 
## Week 5 

Two very important concepts introduced this week around working with data frames are joins and reshaping your data structure. In this notebook we will look at some worked examples of both of these concepts in Python. 

In [ ]:
# as always, first load packages and modules 
import pandas as pd 
import numpy as np

## Joins

In [ ]:
# create a new dataframe called patient_details 
patient_details = pd.DataFrame.from_dict({"id": [1, 2, 3, 4, 5],           
                                          "age": [40, 56, 23, 45, 34]})

patient_details

In [ ]:
# create a new dataframe called appointment details 
## notice only IDs 3, 4, and 5 exist in both data sets 
appt_details = pd.DataFrame.from_dict({"id": [3, 4, 5, 6, 7],
                                       "month": ["March", "May", "June", "June", "April"],
                                       "return_visit": [True, False, True, True, False]})

appt_details

Now lets join these two data sets to create a data set for analysis. 

### Inner join 

In [ ]:
# inner join 

analysis_inner = pd.merge(patient_details, appt_details, 
                          how = "inner", 
                          on = "id")

analysis_inner

After an inner join, only IDs `3`, `4`, and `5` are returned as they are the only matching rows in both data frames. 

### Full join 

In [ ]:
# full join 

analysis_full = pd.merge(patient_details, appt_details, 
                         how = "outer", # full join is denoted with outer  
                         on = "id")

analysis_full

In a full join, as you can see, there are `NaN` values for IDs `1` and `2` in the `month` and `return_visit` columns as these IDs are not in the `appt_details` data frame. And there are `NaN` values for IDs `6` and `7` for in the `age` column as these IDs are not in the `patient_details` data frame. 

### Left join 

In [ ]:
# left join 

analysis_left = pd.merge(patient_details, appt_details, 
                         how = "left", 
                         on = "id")

analysis_left 

In a left join, all of the data from `patient_details` was kept, resulting in `NaN` values for IDs `1` and `2` in the columns `month` and `return_visit` as these IDs are not in the `appt_details` data frame. 

### Right join 

In [ ]:
# right join 

analysis_right = pd.merge(patient_details, appt_details,
                          how = "right", 
                          on = "id")

analysis_right 

A right join is essentially the opposite of a left join, keeping all of the data from `appt_details`. Thus, IDs `6` and `7` have `NaN` for `age` as these IDs are not in the `patient_details` data frame. 

If you switch which data frames are on the right and left, a right join and left join will produce the same result, just with columns in a different order. 

In [ ]:
# left join with order of data frame inputs switched is the same as the right join above 

analysis_left_switch = pd.merge(appt_details, patient_details, 
                                how = "left", 
                                on = "id")

analysis_left_switch 
# which is the same as analysis_right only age is the last column rather than the 2nd

## But what about when the linkage keys are not called the same thing? 

In this case, we need to specify the arguments `left_on` for the left-hand side dataset and `right_on` for the right-hand side dataset. Let's look at a full join as an example. 

In [ ]:
# reminder of what the columns are called in appt details 

appt_details 

In [ ]:
# create a 2nd patient details df with a different name for the id variable 

patient_details2 = pd.DataFrame.from_dict({"patient": [1, 2, 3, 4, 5],           
                                          "age": [40, 56, 23, 45, 34]})

patient_details2

In [ ]:
analysis_full2 = pd.merge(patient_details2, appt_details, 
                          how = "outer", 
                          left_on = "patient", 
                          right_on = "id")

analysis_full2

In [ ]:
analysis_full

As you can see, `analysis_full2` and `analysis_full` are the same in content, though when using `left_on` and `right_on`, both original data frame linked key columns are retained. If you prefer to have the merged data frame with a single ID or linking key column, you can change the name of the variables in the data frames to be merged to match first then perform your merge as above. 

In [ ]:
patient_details2.rename({"patient": "id"}, 
                        axis = 1).merge(appt_details, 
                                        how = "outer", 
                                        on = "id")

# this does not rename patient to id in the patient_details2 df, it just does so for the merge operation
# notice we pass a dictionary structure to the rename method

In [ ]:
# patient_details2 still has ID column called patient
## becuase in the code above we did not change the stored data object 

patient_details2.head()

To merge by multiple keys, you just need to pass the keys in a list to `pd.merge`. 
So for example: `pd.merge(df1, df2, how = "left", on = ["A", "B"])`

## Reshaping data

To move between wide and long data formats in `pandas` we use the `pd.pivot()` and `pd.melt()` functions. 

* `pd.pivot(df, index = , columns = , values =)`: reshape the data from long to wide, where 
    - `columns` are the columns used to make the new data frame's columns 
    - `values` are the columns to use for populating the new data frame's values 
    - `index` is the column(s) used to make the new data frame's index. If not specified, uses the existing index by default 
    
* `pd.melt(df, id_vars = , value_vars = , var_name = , value_name = )`: reshape the data from wide to long, where 
    - `id_vars` are the columns used as identifier variables 
    - `value_vars` are the column(s) to unpivot or make longer
    - `var_name` is the name used for the `variable` column
    - `value_name` is the name used for the `value` column 
    - `ignore_index` if `True`, original index is ignored. If `False`, the original index is retained

To strengthen our understanding of wide and long formats of data, we will again return the `gapminder` data that we have seen a few times before.

In [ ]:
# remember to change the file path to the data to suit your own set up 
gap_data = pd.read_csv("../data/gapminder_data.csv")

gap_data

Sometimes, as with the  gapminder dataset, we have multiple types of observed data. It is somewhere in between the purely long and wide data formats:

* 3 "ID variables": `continent`, `country`, `year`
* 3 "observation variables": `pop` `lifeExp`, `gdpPercap`

It is  pretty common to have data in this format in most cases despite not having ALL observations in 1 column, since all 3 observation variables have different units. Depending on your question being asked of the data, this can be (and often is) considered a tidy format. But we can play with switching it to purely long and wide formats to show what that means (i.e., long would be 4 ID variables and 1 observation variable).


In [ ]:
# make data longer
# notice we are using the list data structure to pass multiple columns to some arguments 

gap_data_long = gap_data.melt(id_vars = ["country", "continent", "year"],
                              value_vars = ["lifeExp", "pop", "gdpPercap"],
                              var_name = "metric")

gap_data_long

In [ ]:
# you do not necessarily need to specify the value_vars parameter 
## but it is good practice to be explicit in your code 

# same outcome as above
gap_data.melt(id_vars = ["country", "continent", "year"],
              var_name = "metric")

Using our new `gap_data_long` data frame, we can convert it back be more like the original `gap_data` format 


In [ ]:
# wide 

gap_data_wide = gap_data_long.pivot(columns = "metric",
                                    values = "value",
                                    index = ["country", "continent", "year"]).reset_index()

gap_data_wide

In [ ]:
# if we remove the reset_index, our resulting df will have a multilevel index
## which could indeed be what you are interested in! 

wide_mindex = gap_data_long.pivot(columns = "metric",
                                  values = "value",
                                  index = ["country", "continent", "year"])

wide_mindex

In [ ]:
## lets compare the indexes from gap_data_wide and wide_mindex 
print(gap_data_wide.index) # range index 

print(wide_mindex.index) # multi index 

We could also make the data even wider by passing multiple columns to the `values` parameter.

In [ ]:
# wider data 

## again without resetting the index we will auotmatically have a MultiIndex 
### with country and continent as we are passing a list to the index parameter 
gap_data_wide2 = gap_data.pivot(columns = "year",
                                values = ["lifeExp", "pop", "gdpPercap"],
                                index = ["country", "continent"])

gap_data_wide2

Let's compare the dimensions of our tibbles: 

In [ ]:
gap_data.shape # 1704 rows and 6 columns 

In [ ]:
gap_data_long.shape  # 5112 rows by 5 columns 

In [ ]:
gap_data_wide.shape # 1704 rows and 6 columns - same as gap_data as expected

In [ ]:
wide_mindex.shape # 1704 rows by 3 columns - becuase we have a MultiIndex with 3 columns! (hence 3 less columns than gap_data_wide)

In [ ]:
gap_data_wide2.shape # 142 rows and 36 columns
## would be 142 rows by 38 columns if we reset the index - try it out and see

As we can see by the dimensions, `gap_data_long` is indeed skinnier (fewer columns) and longer (many more rows) than the other data frames and `gap_data_wide2` is indeed much wider (many more columns and fewer rows)

### Multi-level indexes 

When working with multi-index data frames, we cannot use `df.pivot()` or `df.melt()` but must rather employ the `df.unstack()` and `df.stack()` methods. 

* `df.unstack(level = -1, fill_value = None)` is the MultiIndex equivalent to `df.pivot()`
    - `level` is default, it is set to `-1` (i.e., the last level can be unstacked). If we pass the specified level, it will unstack those levels. 
    - `fill_value` are the values you wish to replace resulting `NaN` values with in the new data frame, by default set to `None` 


* `df.stack(level = -1 , dropna = True)` is the MultiIndex equivalent to `df.melt()`
    - `level` is level(s) to stack from the column axis onto the index axis, defined as one index or label, or a list of indices or labels. By default set to `-1` (i.e., the last level can be stacked)
    - `dropna` is used to manage the rows with resulting `NaN` values. Default is `True` which drops rows in the resulting Frame/Series with all missing values

In [ ]:
# create a same example df with a multi-level index to better see this in action 

multicol1 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('weight', 'pounds')])

df_multi_level_cols1 = pd.DataFrame([[1, 2], [2, 4]],
                                    index = ['cat', 'dog'],
                                    columns = multicol1)

df_multi_level_cols1

In [ ]:
# stack the data frame which has a multi-level column axis 

stack1 = df_multi_level_cols1.stack()

stack1

We can also unstack these columns with the sister function `df.unstack()`

In [ ]:
# unstack 

unstack1 = stack1.unstack()

unstack1

It is common to have missing values when stacking a dataframe with multi-level columns, as the stacked dataframe typically has more values than the original dataframe. Missing values are filled with `NaN`

In [ ]:
multicol2 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                        ('height', 'm')])

df_multi_level_cols2 = pd.DataFrame([[1.0, 2.0], [3.0, 4.0]],
                                     index = ['cat', 'dog'],
                                     columns = multicol2)

df_multi_level_cols2

In [ ]:
stack2 = df_multi_level_cols2.stack()

stack2

In [ ]:
# unstack 
unstack2 = stack2.unstack()

unstack2

In [ ]:
# stack by column axis 0 instead of default -1 

stack3 = df_multi_level_cols2.stack(level = 0)

stack3

In [ ]:
# unstack 

unstack3 = stack3.unstack()

unstack3

In [ ]:
# compare to unstack3 above 

unstack3_2 = stack3.unstack(level = 0)

unstack3_2

---

## You did it! 🎉 

Well done for making it to the end of this notebook. If you have not done so yet, move to the RMarkdwon notebook next. 

⭐⭐⭐❓👣 Do not forget your 3 stars, a wish, and a step mini-diaries once you have completed the content for this week. 

---
*Dr. Brittany Blankinship (2024)*